In [1]:
import pyscfad
from pyscfad import gto,dft,scf
import matplotlib.pyplot as plt
import equinox as eqx
import pyscf
# from pyscf import gto,dft,scf
import numpy as np
import jax.numpy as jnp
import scipy
from ase import Atoms
from ase.io import read
import xcquinox as xce
from functools import partial
from ase.units import Bohr
import os, optax, jax


/home/awills/anaconda3/envs/pyscfad/lib/python3.10/site-packages/pyscf/dft/libxc.py:772: UserWarning: Since PySCF-2.3, B3LYP (and B3P86) are changed to the VWN-RPA variant, the same to the B3LYP functional in Gaussian and ORCA (issue 1480). To restore the VWN5 definition, you can put the setting "B3LYP_WITH_VWN5 = True" in pyscf_conf.py
  warnings.warn('Since PySCF-2.3, B3LYP (and B3P86) are changed to the VWN-RPA variant, '
2024-04-19 17:39:53.084212: E external/xla/xla/stream_executor/cuda/cuda_driver.cc:280] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


In [2]:
PRETRAIN_LEVEL = 'NONLOCAL'

TRAIN_NET = 'x'

REFERENCE_XC = 'PBE0'

N_HIDDEN = 16
DEPTH = 3
if PRETRAIN_LEVEL == 'GGA':
    localx = xce.net.eX(n_input=1, n_hidden=N_HIDDEN, use=[1], depth=DEPTH, lob=1.804)
    localc = xce.net.eC(n_input=3, n_hidden=N_HIDDEN, use=[2], depth=DEPTH, ueg_limit=True)
elif PRETRAIN_LEVEL == 'MGGA':
    localx = xce.net.eX(n_input=2, n_hidden=N_HIDDEN, use=[1, 2], depth=DEPTH, ueg_limit=True, lob=1.174)
    localc = xce.net.eC(n_input=4, n_hidden=N_HIDDEN, depth=DEPTH, use=[2,3], ueg_limit=True)
elif PRETRAIN_LEVEL == 'NONLOCAL':
    localx = xce.net.eX(n_input=18, n_hidden=N_HIDDEN, depth=DEPTH, ueg_limit=True, lob=1.174)
    localc = xce.net.eC(n_input=16, n_hidden=N_HIDDEN, depth=DEPTH, ueg_limit=True)

xc = xce.xc.eXC(grid_models=[localx, localc], heg_mult=True, level= {'GGA':2, 'MGGA':3, 'NONLOCAL':4}[PRETRAIN_LEVEL])

In [3]:
trainms = read('/home/awills/Documents/Research2/torch_dpy/subset09_nf/subat_ref_corrected.traj', ':')
mfs = []
mols = []
energies = []
dms = []
ao_evals = []
gws = []
eris = []
mo_occs = []
hcs = []
vs = []
ts = []
ss = []
hologaps = []
ogds = []
for idx, at in enumerate(trainms[1:2]):
    name, mol = xce.utils.ase_atoms_to_mol(at, basis='def2tzvpd')
    mol.verbose=9
    mol.build()
    mols.append(mol)
    mf = dft.RKS(mol, xc='SCAN')
    e_tot = mf.kernel()
    mfs.append(mf)
    dm = mf.make_rdm1()
    ao_eval = jnp.array(mf._numint.eval_ao(mol, mf.grids.coords, deriv=2))
    energies.append(mf.get_veff().exc)
    dms.append(dm)
    ogds.append(dm.shape)
    ao_evals.append(ao_eval)
    gws.append(mf.grids.weights)
    ts.append(mol.intor('int1e_kin'))
    vs.append(mol.intor('int1e_nuc'))
    mo_occs.append(mf.mo_occ)
    hcs.append(mf.get_hcore())
    eris.append(mol.intor('int2e'))
    ss.append(jnp.linalg.inv(jnp.linalg.cholesky(mol.intor('int1e_ovlp'))))
    hologaps.append(mf.mo_energy[mf.mo_occ == 0][0] - mf.mo_energy[mf.mo_occ > 1][-1])

System: uname_result(system='Linux', node='aegis', release='5.15.0-102-generic', version='#112~20.04.1-Ubuntu SMP Thu Mar 14 14:28:24 UTC 2024', machine='x86_64')  Threads 20
Python 3.10.14 (main, Mar 21 2024, 16:24:04) [GCC 11.2.0]
numpy 1.26.4  scipy 1.11.4
Date: Fri Apr 19 17:39:53 2024
PySCF version 2.3.0
PySCF path  /home/awills/anaconda3/envs/pyscfad/lib/python3.10/site-packages/pyscf

[CONFIG] ARGPARSE = False
[CONFIG] DEBUG = False
[CONFIG] MAX_MEMORY = 4000
[CONFIG] TMPDIR = .
[CONFIG] UNIT = angstrom
[CONFIG] VERBOSE = 3
[CONFIG] conf_file = /home/awills/.pyscf_conf.py
[CONFIG] pyscf_numpy_backend = jax
[CONFIG] pyscf_scipy_backend = jax
[CONFIG] pyscf_scipy_linalg_backend = pyscfad
[CONFIG] pyscfad = True
[CONFIG] pyscfad_ccsd_implicit_diff = True
[CONFIG] pyscfad_scf_implicit_diff = True
[INPUT] verbose = 9
[INPUT] max_memory = 4000 
[INPUT] num. atoms = 2
[INPUT] num. electrons = 14
[INPUT] charge = 0
[INPUT] spin (= nelec alpha-beta = 2S) = 0
[INPUT] symmetry False subgro

/home/awills/anaconda3/envs/pyscfad/lib/python3.10/site-packages/pyscf/gto/mole.py:1215: UserWarning: Function mol.dumps drops attribute coords because it is not JSON-serializable
  warnings.warn(msg)
/home/awills/anaconda3/envs/pyscfad/lib/python3.10/site-packages/pyscf/gto/mole.py:1215: UserWarning: Function mol.dumps drops attribute exp because it is not JSON-serializable
  warnings.warn(msg)
/home/awills/anaconda3/envs/pyscfad/lib/python3.10/site-packages/pyscf/gto/mole.py:1215: UserWarning: Function mol.dumps drops attribute ctr_coeff because it is not JSON-serializable
  warnings.warn(msg)


Padding 4 grids
tot grids = 28096
_block_loop initiating -- parameters
BLKSIZE=56
ni=<pyscfad.dft.numint.NumInt object at 0x7f5e42a21de0>, mol=<pyscfad.gto.mole.Mole object at 0x7f5dd47e3c40>, grids=<pyscf.dft.gen_grid.Grids object at 0x7f5dec1ff9d0>, nao=74, deriv=0, max_memory=4000
non0tab = None, buf=None
_block_loop initiating -- blksize = None
ngrids = 28096
comp = 1
blksize = 28112
Drop grids 2368
    CPU time for setting up grids      2.67 sec, wall time      0.44 sec
_block_loop initiating -- parameters
BLKSIZE=56
ni=<pyscfad.dft.numint.NumInt object at 0x7f5e42a21de0>, mol=<pyscfad.gto.mole.Mole object at 0x7f5dec15af80>, grids=<pyscf.dft.gen_grid.Grids object at 0x7f5dec1ff9d0>, nao=74, deriv=2, max_memory=3443.673088
non0tab = None, buf=None
_block_loop initiating -- blksize = None
ngrids = 25728
comp = 10
blksize = 25760
nelec by numeric integration = 13.999767946791685
    CPU time for vxc      2.40 sec, wall time      0.59 sec
E1 = -194.87176405237446  Ecoul = 75.29872351

/home/awills/anaconda3/envs/pyscfad/lib/python3.10/site-packages/pyscfad/_src/util.py:108: UserWarning: Not taking derivatives wrt the leaves in the node <class 'pyscfad.dft.rks.VXC'> as none of those was specified.
  warnings.warn(f'Not taking derivatives wrt the leaves in '


nelec by numeric integration = 13.999999883825637
    CPU time for vxc      1.97 sec, wall time      0.22 sec
E1 = -194.5180016049281  Ecoul = 75.05000003854624  Exc = -13.656307930840711
cycle= 2 E= -109.525954817647  delta_E= -0.000823  |g|= 0.00764  |ddm|= 0.0811
    CPU time for cycle = 2      2.87 sec, wall time      0.31 sec
getting fock, various matrices
h1e = (74, 74)
s1e=(74, 74)
dm=(74, 74)
diis-norm(errvec)=0.0222635
diis-c [-1.43654424e-04  8.50471132e-02  9.14952887e-01]
fock matrix:  (74, 74)
  HOMO = -0.394409637212138  LUMO = -0.0605814917837219
  mo_energy =
[-14.2670003  -14.26560897  -1.09199458  -0.52495979  -0.43999596
  -0.43999596  -0.39440964  -0.06058149  -0.06058149   0.08861699
   0.14118665   0.27223716   0.28890767   0.28893923   0.30123888
   0.30123888   0.35383764   0.35383764   0.39483894   0.48250399
   0.48264143   0.48944645   0.48944645   0.51505355   0.53935849
   0.72001728   0.72049323   0.80582916   0.80582916   1.08136324
   1.10488238   1.1049

In [4]:
e_tot

Array(-109.52596483, dtype=float64)

In [5]:
def generate_network_eval_xc(mf, dm, ao, gw, network):
    def eval_xc(xc_code, rho, spin=0, relativity=0, deriv=1, omega=None, verbose=None, ao=ao, weight=gw, coords=None):
        print('custom eval_xc; input rho shape: ', rho.shape)
        if len(rho.shape) == 2:
            rho0 = rho[0]
            drho = rho[1:4]
            tau = 0.5*(rho[1] + rho[2] + rho[3])
            
            non_loc = jnp.zeros_like(tau)
        print(f'decomposed shapes:\nrho0={rho0.shape}\ndrho={drho.shape}\ntau={tau.shape}\nnon_loc={non_loc.shape}')
        if dm.ndim == 3: # If unrestricted (open-shell) calculation

            # Density
            rho0_a = rho0[0]
            rho0_b = rho0[1]

            # jnp.einsumed density gradient
            gamma_a, gamma_b = jnp.einsum('ij,ij->j',drho[:,0],drho[:,0]), jnp.einsum('ij,ij->j',drho[:,1],drho[:,1])
            gamma_ab = jnp.einsum('ij,ij->j',drho[:,0],drho[:,1])

            # Kinetic energy density
            tau_a, tau_b = tau

            # E.-static
            non_loc_a, non_loc_b = non_loc
        else:
            rho0_a = rho0_b = rho0*0.5
            gamma_a=gamma_b=gamma_ab= jnp.einsum('ij,ij->j',drho[:],drho[:])*0.25
            tau_a = tau_b = tau*0.5
            non_loc_a=non_loc_b = non_loc*0.5

        # xc-energy per unit particle
        print(f'EVALUATING GRID MODELS; OPTIONAL PARAMETERS:')
        try:
            print(f'gw.shape={gw.shape}, coor.shape={coor.shape}')
        except:
            print('no externally supplied gw or coor')
        exc = network.eval_grid_models(jnp.concatenate([jnp.expand_dims(rho0_a,-1),
                                                jnp.expand_dims(rho0_b,-1),
                                                jnp.expand_dims(gamma_a,-1),
                                                jnp.expand_dims(gamma_ab,-1),
                                                jnp.expand_dims(gamma_b,-1),
                                                jnp.expand_dims(jnp.zeros_like(rho0_a),-1), #Dummy for laplacian
                                                jnp.expand_dims(jnp.zeros_like(rho0_a),-1), #Dummy for laplacian
                                                jnp.expand_dims(tau_a,-1),
                                                jnp.expand_dims(tau_b,-1),
                                                jnp.expand_dims(non_loc_a,-1),
                                                jnp.expand_dims(non_loc_b,-1)],axis=-1),
                                   mf = mf, dm = dm, ao=ao, gw=weight, coor=coords)
        exc = exc[:, 0]
        print('exc from network evaluation on grid models shape: ', exc.shape)
        vgf = lambda x: network(x, ao, weight, mf=mf, coor=coords)
        mf.network = xc
        mf.network_eval = vgf
        print('finding vxc from jax.grad(vgf)(dm)')
        vxc = jax.grad(vgf)(dm)
        # vrho = vxc
        vrho = exc*rho0
        vgamma = jnp.zeros_like(vrho)
        vlapl = None
        vtau = jnp.nan_to_num(exc*tau)
        fxc = None #second order functional derivative
        kxc = None #third order functional derivative
        print(f'shapes: vxc={vxc.shape}, vrho={vrho.shape}, vgamma={vgamma.shape}')
        return exc, (vrho, vgamma, vlapl, vtau), fxc, kxc
    return eval_xc

In [6]:
mf.mol.spin

0

In [7]:
# evxc = generate_network_eval_xc(mf=mf, dm=dm, ao=ao_eval, gw=mf.grids.weights, network=xc)

In [8]:
# mf.define_xc_(evxc, xctype='MGGA')

In [9]:
# mf.kernel()

In [10]:
class E_PySCFAD_loss(eqx.Module):
    def __init__(self):
        '''
        The standard energy loss module, RMSE loss of predicted vs. reference energies.
        '''
        super().__init__()

    def __call__(self, model, mf, inp_dm, ao, gw, ref_en):
        '''
        Computes the energy loss for a given model and associated input density matrix, atomic orbitals on the grid, and grid weights

        Loss is the RMSE energy, so predicted energy can potentially be a jax.Array of SCF guesses.

        :param model: The XC object whose forward pass predicts the XC energy based on the inputs here.
        :type model: xcquinox.xc.eXC
        :param inp_dm: The density matrix to pass into the network for density creation on the grid.
        :type inp_dm: jax.Array
        :param ref_en: The reference energy to take the loss with respect to.
        :type ref_en: jax.Array
        :param ao_eval: Atomic orbitals evaluated on the grid
        :type ao_eval: jax.Array
        :param grid_weights: pyscfad's grid weights for the reference calculation
        :type grid_weights: jax.Array
        :return: The RMSE error.
        :rtype: jax.Array
        '''
        print('generating eval_xc function to overwrite')
        vgf = lambda x: model(x, ao, gw, mf=mf)
        mf.network = model
        mf.network_eval = vgf
        mf.max_memory=16000
        evxc = generate_network_eval_xc(mf=mf, dm=inp_dm, ao=ao, gw=gw, network=model)
        mf.define_xc_(evxc, xctype='MGGA')
        print('predicting energy...')
        e_pred = mf.kernel()
        print('energy predicted')
        eL = jnp.sqrt( jnp.mean((e_pred-ref_en)**2))
        return eL


In [11]:
cpus = jax.devices(backend='cpu')

In [ ]:
scheduler = optax.exponential_decay(init_value = 1e-2, transition_begin=50, transition_steps=500, decay_rate=0.9)
optimizer = optax.adam(learning_rate = 1e-2)
# optimizer = optax.adam(learning_rate = scheduler)

trainer = xce.train.xcTrainer(model=xc, optim=optimizer, steps=500, loss = E_PySCFAD_loss(), do_jit=False)
with jax.default_device(cpus[0]):
    newm = trainer(1, trainer.model, mfs, dms, ao_evals, gws, [-109.52596483])


Epoch 0
Epoch 0 :: Batch 0/1
generating eval_xc function to overwrite
predicting energy...
Set gradient conv threshold to 3.16228e-05
Initial guess from minao.
cond(S) = 198544.46318793358
_block_loop initiating -- parameters
BLKSIZE=56
ni=<pyscfad.dft.numint.NumInt object at 0x7f5e42a21de0>, mol=<pyscfad.gto.mole.Mole object at 0x7f5dec15af80>, grids=<pyscf.dft.gen_grid.Grids object at 0x7f5dec1ff9d0>, nao=74, deriv=2, max_memory=14773.522432
non0tab = None, buf=None
_block_loop initiating -- blksize = None
ngrids = 25728
comp = 10
blksize = 25760


<class 'pyscfad.dft.rks.RKS'> does not have attributes  network_eval network
/home/awills/anaconda3/envs/pyscfad/lib/python3.10/site-packages/pyscf/gto/mole.py:1215: UserWarning: Function mol.dumps drops attribute coords because it is not JSON-serializable
  warnings.warn(msg)
/home/awills/anaconda3/envs/pyscfad/lib/python3.10/site-packages/pyscf/gto/mole.py:1215: UserWarning: Function mol.dumps drops attribute exp because it is not JSON-serializable
  warnings.warn(msg)
/home/awills/anaconda3/envs/pyscfad/lib/python3.10/site-packages/pyscf/gto/mole.py:1215: UserWarning: Function mol.dumps drops attribute ctr_coeff because it is not JSON-serializable
  warnings.warn(msg)


custom eval_xc; input rho shape:  (6, 25728)
decomposed shapes:
rho0=(25728,)
drho=(3, 25728)
tau=(25728,)
non_loc=(25728,)
EVALUATING GRID MODELS; OPTIONAL PARAMETERS:
no externally supplied gw or coor
custom gw and coor present in eval_grid_models; shapes: gw=(25728,), coor=(25728, 3)
RKSAnalyzer; coor specified, shape =  (25728, 3)
RKSAnalyzer; weight specified, shape =  (25728,)
ElectronAnalyzer; coor specified, shape =  (25728, 3)
RKSAnalyzer; weight specified, shape =  (25728,)


/home/awills/anaconda3/envs/pyscfad/lib/python3.10/site-packages/jax/_src/ops/scatter.py:96: FutureWarning: scatter inputs have incompatible types: cannot safely cast value from dtype=int64 to dtype=int32 with jax_numpy_dtype_promotion='standard'. In future JAX releases this will result in an error.
  warnings.warn(


nl_4, descr5 shape: (12, 25728)
descr shape:  (25728, 4)
descr5 shape:  (25728, 12)
RKSAnalyzer; coor specified, shape =  (25728, 3)
RKSAnalyzer; weight specified, shape =  (25728,)
ElectronAnalyzer; coor specified, shape =  (25728, 3)
RKSAnalyzer; weight specified, shape =  (25728,)
nl_4, descr5 shape: (12, 25728)
descr shape:  (25728, 6)
descr5 shape:  (25728, 12)
eX.__call__, rho shape: (25728, 18)
eC.__call__, rho shape: (25728, 16)
exc from network evaluation on grid models shape:  (25728,)
finding vxc from jax.grad(vgf)(dm)
custom gw and coor present in eval_grid_models; shapes: gw=(25728,), coor=(25728, 3)
RKSAnalyzer; coor specified, shape =  (25728, 3)
RKSAnalyzer; weight specified, shape =  (25728,)
ElectronAnalyzer; coor specified, shape =  (25728, 3)
RKSAnalyzer; weight specified, shape =  (25728,)


...

nl_4, descr5 shape: (12, 224)
descr shape:  (224, 4)
descr5 shape:  (224, 12)
RKSAnalyzer; coor specified, shape =  (224, 3)
RKSAnalyzer; weight specified, shape =  (224,)
ElectronAnalyzer; coor specified, shape =  (224, 3)
RKSAnalyzer; weight specified, shape =  (224,)
nl_4, descr5 shape: (12, 224)
descr shape:  (224, 6)
descr5 shape:  (224, 12)
eX.__call__, rho shape: (224, 18)
eC.__call__, rho shape: (224, 16)
exc from network evaluation on grid models shape:  (224,)
finding vxc from jax.grad(vgf)(dm)
custom gw and coor present in eval_grid_models; shapes: gw=(224,), coor=(224, 3)
RKSAnalyzer; coor specified, shape =  (224, 3)
RKSAnalyzer; weight specified, shape =  (224,)
ElectronAnalyzer; coor specified, shape =  (224, 3)
RKSAnalyzer; weight specified, shape =  (224,)


/home/awills/anaconda3/envs/pyscfad/lib/python3.10/site-packages/jax/_src/ops/scatter.py:96: FutureWarning: scatter inputs have incompatible types: cannot safely cast value from dtype=int64 to dtype=int32 with jax_numpy_dtype_promotion='standard'. In future JAX releases this will result in an error.
  warnings.warn(


nl_4, descr5 shape: (12, 224)
descr shape:  (224, 4)
descr5 shape:  (224, 12)
RKSAnalyzer; coor specified, shape =  (224, 3)
RKSAnalyzer; weight specified, shape =  (224,)
ElectronAnalyzer; coor specified, shape =  (224, 3)
RKSAnalyzer; weight specified, shape =  (224,)
nl_4, descr5 shape: (12, 224)
descr shape:  (224, 6)
descr5 shape:  (224, 12)
eX.__call__, rho shape: (224, 18)
eC.__call__, rho shape: (224, 16)
shapes: vxc=(74, 74), vrho=(224,), vgamma=(224,)
custom eval_xc; input rho shape:  (6, 224)
decomposed shapes:
rho0=(224,)
drho=(3, 224)
tau=(224,)
non_loc=(224,)
EVALUATING GRID MODELS; OPTIONAL PARAMETERS:
no externally supplied gw or coor
custom gw and coor present in eval_grid_models; shapes: gw=(224,), coor=(224, 3)
RKSAnalyzer; coor specified, shape =  (224, 3)
RKSAnalyzer; weight specified, shape =  (224,)
ElectronAnalyzer; coor specified, shape =  (224, 3)
RKSAnalyzer; weight specified, shape =  (224,)


/home/awills/anaconda3/envs/pyscfad/lib/python3.10/site-packages/jax/_src/ops/scatter.py:96: FutureWarning: scatter inputs have incompatible types: cannot safely cast value from dtype=int64 to dtype=int32 with jax_numpy_dtype_promotion='standard'. In future JAX releases this will result in an error.
  warnings.warn(


nl_4, descr5 shape: (12, 224)
descr shape:  (224, 4)
descr5 shape:  (224, 12)
RKSAnalyzer; coor specified, shape =  (224, 3)
RKSAnalyzer; weight specified, shape =  (224,)
ElectronAnalyzer; coor specified, shape =  (224, 3)
RKSAnalyzer; weight specified, shape =  (224,)
nl_4, descr5 shape: (12, 224)
descr shape:  (224, 6)
descr5 shape:  (224, 12)
eX.__call__, rho shape: (224, 18)
eC.__call__, rho shape: (224, 16)
exc from network evaluation on grid models shape:  (224,)
finding vxc from jax.grad(vgf)(dm)
custom gw and coor present in eval_grid_models; shapes: gw=(224,), coor=(224, 3)
RKSAnalyzer; coor specified, shape =  (224, 3)
RKSAnalyzer; weight specified, shape =  (224,)
ElectronAnalyzer; coor specified, shape =  (224, 3)
RKSAnalyzer; weight specified, shape =  (224,)


Modifications in mldftdat, pyscfad, pyscf, xcquinox. 

xcquinox modifications took place in package repo, so no need to find differences

mldftdat and pyscfad changes occured in package directory, need to find differences for mldftdat to put into xcquinox-cider and a patch for pyscfad